In [20]:
base_dir = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/s-mffroque1/code/Users/s-mffroque/image_splits_MELandBEN"

train_dir = os.path.join(base_dir, 'Training')
test_dir = os.path.join(base_dir, 'Testing')
validate_dir = os.path.join(base_dir, 'validate')

In [4]:
os.getcwd()

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/s-mffroque1/code/Users/s-mffroque'

In [24]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
import os
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


img_size=150

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=5,
        zoom_range = 0.1 ,
        horizontal_flip=False,
        channel_shift_range=.001
        #preprocessing_function= lambda x: tf.image.rgb_to_hsv(x)
    )

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_size, img_size),
                      batch_size=8,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validate_dir,
        target_size=(img_size, img_size),
        batch_size=8,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_size, img_size),
        batch_size=8,
        class_mode='binary')


model = models.Sequential()
model.add(layers.Conv2D(128, (5, 5), activation='relu', input_shape=(img_size, img_size, 3)))
model.add(layers.Conv2D(128, (5, 5), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(1024, (3, 3), activation='relu'))
model.add(layers.Conv2D(1024, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(2048, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(2048, activation="relu"))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(1024, activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.Dense(1, activation='sigmoid'))

from tensorflow.keras import metrics
opt = tf.keras.optimizers.Adam(
        learning_rate=2e-5,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=True,
        name="Adam")

model.compile(optimizer=opt,
                  loss='binary_crossentropy',
                  metrics=[metrics.Recall(),metrics.Precision(),'accuracy', metrics.AUC(name = 'AUC'),
                       metrics.AUC(curve='PR', name = 'PR-AUC'),metrics.FalsePositives(),
                       metrics.FalseNegatives(), metrics.TrueNegatives(),metrics.TruePositives()])



history = model.fit(
        train_generator,
        epochs=50,
        validation_data=validation_generator,
                       verbose=1)
#model.save('supermodel_final.h5')


Num GPUs Available:  1
Found 12166 images belonging to 2 classes.
Found 1539 images belonging to 2 classes.
Found 1521 images belonging to 2 classes.
Epoch 1/50
1521/1521 [==============================] - 1710s 1s/step - loss: 0.6303 - recall_15: 0.6351 - precision_15: 0.6359 - accuracy: 0.6522 - AUC: 0.7045 - PR-AUC: 0.6577 - false_positives_15: 2112.0000 - false_negatives_15: 2119.0000 - true_negatives_15: 4247.0000 - true_positives_15: 3688.0000 - val_loss: 0.7795 - val_recall_15: 0.9907 - val_precision_15: 0.4973 - val_accuracy: 0.5049 - val_AUC: 0.7750 - val_PR-AUC: 0.7423 - val_false_positives_15: 755.0000 - val_false_negatives_15: 7.0000 - val_true_negatives_15: 30.0000 - val_true_positives_15: 747.0000
Epoch 2/50
1521/1521 [==============================] - 1772s 1s/step - loss: 0.5888 - recall_15: 0.6620 - precision_15: 0.6801 - accuracy: 0.6900 - AUC: 0.7545 - PR-AUC: 0.7061 - false_positives_15: 1808.0000 - false_negatives_15: 1963.0000 - true_negatives_1

Bad pipe message: %s [b'i\xd6\x14\t\xf9\x96}\xa1\x16\x88r#\xadM|\xf3\xaf\xc2 \xc3<\xf4\xe6\xe7\xab\xc2d\xb1\xe5\xe3\xb6\x0cK\xedZ\xd6\xcc']
Bad pipe message: %s [b"\xdb\xf7s\x12\xe7m\x80!\x0c\xcfc\xc4\xaa\xa1\x9f\xef'\xf5\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f", b"\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x000\x00.\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06", b'', b'\x03\x03']
Bad pipe message: %s [b'']
Bad pi

In [25]:
model.save('/mnt/batch/tasks/shared/LS_root/mounts/clusters/s-mffroque1/code/Users/s-mffroque/models/gigamodel_noHSV_50epoch.h5')

In [27]:
from azureml.core import Workspace
from azureml.core.compute import AmlCompute
ws = Workspace.from_config()
compute_target = ws.compute_targets['s-mffroque1']

compute_target
compute_target.stop(wait_for_completion=True)

In [1]:
ws = Workspace.from_config()
compute_target = ws.compute_targets['s-mffroque1']
compute_target.start()


NameError: name 'Workspace' is not defined

In [1]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
import os
import cv2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

base_dir = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/s-mffroque1/code/Users/s-mffroque/image_splits_MELandBEN"

train_dir = os.path.join(base_dir, 'Training')
test_dir = os.path.join(base_dir, 'Testing')
validate_dir = os.path.join(base_dir, 'validate')

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
"""
def apply_clahe(img):
    # Convert the image to LAB color space
    lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

    # Split the LAB image into its 3 channels
    l_channel, a_channel, b_channel = cv2.split(lab_img)

    # Apply CLAHE to the L channel
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    l_channel = clahe.apply(l_channel)

    # Merge the CLAHE-enhanced L channel with the A and B channels
    lab_img = cv2.merge((l_channel, a_channel, b_channel))

    # Convert the LAB image back to RGB color space
    clahe_img = cv2.cvtColor(lab_img, cv2.COLOR_LAB2BGR)

    return clahe_img
"""
img_size=512
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
    channel_shift_range=0.001
)


test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_size, img_size),
                      batch_size=8,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validate_dir,
        target_size=(img_size, img_size),
        batch_size=8,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_size, img_size),
        batch_size=8,
        class_mode='binary')

regularizer = l2(0.001)


model = models.Sequential()

model.add(layers.Conv2D(32, (5, 5), activation='relu', input_shape=(img_size, img_size, 3)))
model.add(layers.Conv2D(32, (5, 5), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu',kernel_regularizer=regularizer))
model.add(layers.Conv2D(64, (3, 3), activation='relu',kernel_regularizer=regularizer))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu',kernel_regularizer=regularizer))
model.add(layers.Conv2D(128, (3, 3), activation='relu',kernel_regularizer=regularizer))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu',kernel_regularizer=regularizer))
model.add(layers.Conv2D(256, (3, 3), activation='relu',kernel_regularizer=regularizer))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(512, (3, 3), activation='relu',kernel_regularizer=regularizer))
model.add(layers.Conv2D(512, (3, 3), activation='relu',kernel_regularizer=regularizer))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(1024, (3, 3), activation='relu',kernel_regularizer=regularizer))
model.add(layers.Conv2D(1024, (3, 3), activation='relu',kernel_regularizer=regularizer))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(2048, (3, 3), activation='relu',kernel_regularizer=regularizer))
model.add(layers.BatchNormalization())

model.add(layers.MaxPooling2D((2, 2)))


model.add(layers.Flatten())
model.add(layers.Dense(2048, activation="relu",kernel_regularizer=regularizer))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(1024, activation="relu",kernel_regularizer=regularizer))
model.add(layers.BatchNormalization())
model.add(layers.Dense(1, activation='sigmoid',kernel_regularizer=regularizer))
from keras.callbacks import ModelCheckpoint

from tensorflow.keras import metrics
opt = tf.keras.optimizers.Adam(
        learning_rate=2e-5,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=True,
        name="Adam")

checkpoint = ModelCheckpoint('checkpoint_best_gigamodel_superchanged_20epoch_512imagesize.h5', monitor='val_recall_3', verbose=1,
                             save_best_only=True, mode='max')

callbacks_list = [checkpoint]

model.compile(optimizer=opt,
                  loss='binary_crossentropy',
                  metrics=[metrics.Recall(),metrics.Precision(),'accuracy', metrics.AUC(name = 'AUC'),metrics.AUC(curve='PR', name = 'PR-AUC')
                       ,metrics.FalsePositives(), metrics.FalseNegatives(), metrics.TrueNegatives(),metrics.TruePositives()])



history = model.fit(
        train_generator,
        epochs=20,
        validation_data=validation_generator,callbacks=[callbacks_list],
                       verbose=1)


2023-04-09 12:29:16.086137: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-09 12:29:40.731412: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-09 12:29:40.731526: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-04-09 12:29:40.731537: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

Num GPUs Available:  1
Found 12166 images belonging to 2 classes.
Found 1539 images belonging to 2 classes.
Found 1521 images belonging to 2 classes.
Epoch 1/20
1521/1521 [==============================] - 2356s 1s/step - loss: 8.7203 - recall: 0.6625 - precision: 0.6269 - accuracy: 0.6507 - AUC: 0.7036 - PR-AUC: 0.6425 - false_positives: 2290.0000 - false_negatives: 1960.0000 - true_negatives: 4069.0000 - true_positives: 3847.0000 - val_loss: 8.5846 - val_recall: 0.6552 - val_precision: 0.6823 - val_accuracy: 0.6816 - val_AUC: 0.7508 - val_PR-AUC: 0.6947 - val_false_positives: 230.0000 - val_false_negatives: 260.0000 - val_true_negatives: 555.0000 - val_true_positives: 494.0000
Epoch 2/20
1521/1521 [==============================] - 2202s 1s/step - loss: 8.5554 - recall: 0.6787 - precision: 0.6426 - accuracy: 0.6664 - AUC: 0.7296 - PR-AUC: 0.6715 - false_positives: 2192.0000 - false_negatives: 1866.0000 - true_negatives: 4167.0000 - true_positives: 3941.0000 - val_loss: 8.4097 - val_r

In [2]:
model.save('/mnt/batch/tasks/shared/LS_root/mounts/clusters/s-mffroque1/code/Users/s-mffroque/models/gigamodel_superchanged_20epoch_512imagesize.h5')

In [4]:
from azureml.core import Workspace
from azureml.core.compute import AmlCompute
ws = Workspace.from_config()
compute_target = ws.compute_targets['s-mffroque1']

compute_target
compute_target.stop(wait_for_completion=True)

In [7]:
from tensorflow.keras import models
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import os
base_dir = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/s-mffroque1/code/Users/s-mffroque/image_splits_MELandBEN"

train_dir = os.path.join(base_dir, 'Training')
test_dir = os.path.join(base_dir, 'Testing')
validate_dir = os.path.join(base_dir, 'validate')
img_size=224

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,
    zoom_range = 0.1 ,
    horizontal_flip=False,
    channel_shift_range=.001)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
                  batch_size=16,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validate_dir,
    target_size=(img_size, img_size),
    batch_size=16,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=16,
    class_mode='binary')


model = models.Sequential()
model.add(layers.Conv2D(64, (5, 5), activation='relu', input_shape=(img_size, img_size, 3)))
model.add(layers.Conv2D(64, (5, 5), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(1024, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.Dense(1, activation='sigmoid'))

from tensorflow.keras import metrics
opt = tf.keras.optimizers.Adam(
    learning_rate=2e-5,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=True,
    name="Adam")
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('bestmodelsuperbasic_30epoc.h5', monitor='val_recall_2', verbose=1,
                             save_best_only=True, mode='max')

callbacks_list = [checkpoint]

model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=[metrics.Recall(),metrics.Precision(),'accuracy', metrics.AUC(name = 'AUC'),metrics.AUC(curve='PR', name = 'PR-AUC')
                       ,metrics.FalsePositives(), metrics.FalseNegatives(), metrics.TrueNegatives(),metrics.TruePositives()])


history = model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator, verbose=1,callbacks=callbacks_list)

Found 12166 images belonging to 2 classes.
Found 1539 images belonging to 2 classes.
Found 1521 images belonging to 2 classes.
Epoch 1/30
761/761 [==============================] - ETA: 0s - loss: 0.5980 - recall_2: 0.6921 - precision_2: 0.6520 - accuracy: 0.6767 - AUC: 0.7414 - PR-AUC: 0.6953 - false_positives_2: 2145.0000 - false_negatives_2: 1788.0000 - true_negatives_2: 4214.0000 - true_positives_2: 4019.0000
Epoch 1: val_recall_2 improved from -inf to 0.54111, saving model to bestmodelsuperbasic_30epoc.h5
761/761 [==============================] - 1768s 2s/step - loss: 0.5980 - recall_2: 0.6921 - precision_2: 0.6520 - accuracy: 0.6767 - AUC: 0.7414 - PR-AUC: 0.6953 - false_positives_2: 2145.0000 - fals

In [9]:
model.save('/mnt/batch/tasks/shared/LS_root/mounts/clusters/s-mffroque1/code/Users/s-mffroque/models/bestmodelsuperbasic_30epoc.h5')

In [10]:
from tensorflow.keras import models
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import os
base_dir = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/s-mffroque1/code/Users/s-mffroque/image_splits_MELandBEN"

train_dir = os.path.join(base_dir, 'Training')
test_dir = os.path.join(base_dir, 'Testing')
validate_dir = os.path.join(base_dir, 'validate')
img_size=224

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,
    zoom_range = 0.1 ,
    horizontal_flip=False,
    channel_shift_range=.001)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
                  batch_size=16,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validate_dir,
    target_size=(img_size, img_size),
    batch_size=16,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=16,
    class_mode='binary')

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import VGG16

# create an instance of the ResNet50 model
base_model = base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

model2 = models.Sequential()
model2.add(base_model)
model2.add(layers.Flatten())
model2.add(layers.Dense(2048, activation="relu"))
model2.add(layers.Dropout(0.5))

model2.add(layers.Dense(256, activation="relu"))
model2.add(layers.Dropout(0.5))
model2.add(layers.BatchNormalization())
model2.add(layers.Dense(1, activation='sigmoid'))

from tensorflow.keras import metrics
opt = tf.keras.optimizers.Adam(
    learning_rate=2e-5,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=True,
    name="Adam")
from keras.callbacks import ModelCheckpoint
    
checkpoint = ModelCheckpoint('bestmodeltflearning_vgg.h5', monitor='val_recall_2', verbose=1,
                             save_best_only=True, mode='max')

callbacks_list = [checkpoint]

model2.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=[metrics.Recall(),metrics.Precision(),'accuracy', metrics.AUC(name = 'AUC'),metrics.AUC(curve='PR', name = 'PR-AUC')
                       ,metrics.FalsePositives(), metrics.FalseNegatives(), metrics.TrueNegatives(),metrics.TruePositives()])


history = model2.fit(
    train_generator,
    epochs=25,
    validation_data=validation_generator, verbose=1,callbacks=callbacks_list)

Found 12166 images belonging to 2 classes.
Found 1539 images belonging to 2 classes.
Found 1521 images belonging to 2 classes.
58889256/58889256 [==============================] - 2s 0us/step
Epoch 1/25
761/761 [==============================] - 1747s 2s/step - loss: 0.6852 - recall_3: 0.6282 - precision_3: 0.6180 - accuracy: 0.6372 - AUC: 0.6887 - PR-AUC: 0.6713 - false_positives_3: 2255.0000 - false_negatives_3: 2159.0000 - true_negatives_3: 4104.0000 - true_positives_3: 3648.0000 - val_loss: 0.5203 - val_recall_3: 0.6114 - val_precision_3: 0.8074 - val_accuracy: 0.7381 - val_AUC: 0.8299 - val_PR-AUC: 0.8225 - val_false_positives_3: 110.0000 - val_false_negatives_3: 293.0000 - val_true_negatives_3: 675.0000 - val_true_positives_3: 461.0000
Epoch 2/25
761/761 [==============================] - 1706s 2s/step - loss: 0.6121 - recall_3: 0.6683 - precision_3: 0.6768 - accuracy: 0.6894 - AUC: 0.7560 - PR-AUC: 0.7428 - false_positives_